<a href="https://colab.research.google.com/github/firdaaacy/Praktikum-PPM/blob/main/Praktikum%204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 4. Klasifikasi Naive Bayes


### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [141]:
#mengunduh dataset yang akan digunakan
!wget https://dataset-ppm.s3.amazonaws.com/iris.csv

--2021-07-09 13:55:24--  https://dataset-ppm.s3.amazonaws.com/iris.csv
Resolving dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)... 52.216.176.51
Connecting to dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)|52.216.176.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4609 (4.5K) [text/csv]
Saving to: ‘iris.csv.4’

iris.csv.4          100%[===================>]   4.50K  --.-KB/s    in 0s      

2021-07-09 13:55:25 (195 MB/s) - ‘iris.csv.4’ saved [4609/4609]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [142]:
#melakukan import library yang akan digunakan
import pandas as pd #pemanggilan library pandas dengan alias pemanggilan pd
import numpy as np #pemanggilan library numpy dengan alias pemanggilan np

#melakukan pembacaan dataset yang sudah diunduh menggunakan pandas
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [143]:
#menampilkan 5 data teratas
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [144]:
#melakukan import sebagian library yang akan digunakan
#pada kasus  ini hanya mengimport train_test_split dari modul sklearn.model_selection
from sklearn.model_selection import train_test_split
#melakukan pembagian dataset dengan pembagian 20% data uji dan 80% data latih
data_latih, data_uji = train_test_split(data, test_size = 0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [145]:
#menampilkan jumlah baris data uji yang sudah dibagi 20% dari dataset
print(data_uji.shape[0])
#menampilkan jumlah baris data latih yang sudah dibagi 80% dari dataset
print(data_latih.shape[0])

30
120


Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_uji**

In [146]:
#melakukan pemisahan antara fitur dan label dengan melakukan pop pada kolom spesies pada label latih
data_latih1 = data_latih.copy()
label_latih = data_latih1.pop('species')
#melakukan pemisahan antara fitur dan label dengan melakukan pop pada kolom spesies pada label uji
data_uji1 = data_uji.copy()
label_uji = data_uji1.pop('species')

In [147]:
#menampilkan data uji yang hanya berisikan fitur data saja tanpa label
data_uji1.head()

,sepal_length,sepal_width,petal_length,petal_width
84,5.4,3.0,4.5,1.5
49,5.0,3.3,1.4,0.2
9,4.9,3.1,1.5,0.1
102,7.1,3.0,5.9,2.1
100,6.3,3.3,6.0,2.5


In [148]:
#menampilkan data latih yang hanya berisikan fitur data saja tanpa label
data_latih1.head()

,sepal_length,sepal_width,petal_length,petal_width
18,5.7,3.8,1.7,0.3
119,6.0,2.2,5.0,1.5
129,7.2,3.0,5.8,1.6
53,5.5,2.3,4.0,1.3
135,7.7,3.0,6.1,2.3


## 3) Menghitung Prior

Tahapan pertama pada algoritma Naive Bayes adalah perhitungan prior. Prior suatu kelas merupakan peluang awal munculnya kelas tersebut. Atau dengan kata lain, prior merupakan frekuensi relatif dari suatu kelas terhadap keseluruhan data. 

Buatlah fungsi bernama **hitung_prior** yang berfungsi menghitung nilai prior dari masing-masing kelas yang terdapat pada data latih. Fungsi ini menerima input berupa list kelas dari data latih.

In [149]:
#melakukan import library counter dari library collections
from collections import Counter

#membuat fungsi hitung_prior dengan argumen list kelas
def hitung_prior (list_kelas):
  #mencari jumlah keseluruhan kelas pada list
  n_data = len(list_kelas)
  #menghitung jumlah data yang memiliki kelas yang sama
  prior = Counter(list_kelas)
  #menghitung probabilitas 1 kelas dalam dataset
  for key in prior.keys():
    prior[key] = prior[key]/n_data
  return prior


Ujilah fungsi **hitung_prior** menggunakan kelas dari variabel **data_latih**. Amati hasilnya

In [150]:
#melakukan penghitungan probabilitas prior berdasarkan kelas setiap data
prior = hitung_prior(data_latih['species'])
print(prior)

Counter({'Iris-versicolor': 0.35833333333333334, 'Iris-virginica': 0.325, 'Iris-setosa': 0.31666666666666665})


## 4) Menghitung *likelihood*

Praktikum kali ini mengimplementasikan metode Naive Bayes menggunakan datset iris. Semua fitur pada dataset iris memiliki tipe numerik. Dengan demikian, perhitungan *likelihood* dilakukan menggunakan Gaussian Model.

Tahapan yang perlu dilakukan sebelum perhitungan *likelihood* adalah menghitung rata-rata dan deviasi standar masing-masing fitur per kelas. Buatlah fungsi **hitung_rata2_std_kelas** yang berfungsi menghitung rata-rata dan standar deviasi per fitur dan kelas

In [151]:
#membuat fungsi hitung_rata2_std_kelas dengan argumen list kelas
def hitung_rata2_std_kelas(input_data):
  #membuat list kolom yang berisi fitur dataset
  list_columns = input_data.columns[:-1]
  #membuat list kolom yang berisi kelas/label dataset
  class_column_name = input_data.columns[-1]
  #membuat set data yang berisikan kelas tiap data dan tidak ada duplikat
  list_class = set(input_data[class_column_name])
  #membuat dictionary kosong
  rata2 = {}
  #membuat dictionary kosong
  std = {}
  for column in list_columns:
    for a_class in list_class:
      #mengitung rata-rata tiap kolom pada tiap kelas
      rata2[(a_class,column)] = input_data.loc[input_data[class_column_name]==a_class][column].mean()
      #mengitung standar deviasi tiap kolom pada tiap kelas
      std[(a_class,column)] = input_data.loc[input_data[class_column_name]==a_class][column].std()
  
  return(rata2,std)

Buatlah fungsi **hitung_likelihood_gaussian** yang bertujuan menghitung nilai likelihood suatu nilai terhadap rata-rata dan deviasi standar tertentu.

In [152]:
rata2,std = hitung_rata2_std_kelas(data_latih)

In [153]:
#melakukan import library math
import math

#membuat fungsi hitung_likelihood_gaussian dengan 3 argumen yakni data, rata-rata, dan standar deviasi
def hitung_likelihood_gaussian(data, rata2,std):
  hasil = (1/math.sqrt(2*math.pi*(std**2)))*math.exp((-1*((data-rata2)**2))/(2*(std**2)))
  return hasil

## 5) Proses *training*
Proses training pada Naive Bayes dengan model Gaussian dilakukan untuk menghitung prior serta  rata-rata dan deviasi standar pada masing-msaing fitur dan kelas. 

In [154]:
#membuat fungsi training naive bayes
def training_naive_bayes(data_latih):
  #membuat list kolom yang berisi kelas/label dataset
  class_column_name = data_latih.columns[-1]
  #menghitung probabilitas prior
  prior = hitung_prior(data_latih[class_column_name])
  #menghitung rata-rata dan standar deviasi data latih
  (rata2, std) = hitung_rata2_std_kelas(data_latih)
  #membuat set data yang berisikan kelas tiap data dan tidak ada duplikat
  list_class = set(data_latih[class_column_name])
  #mengambil semua fitur pada data latih
  list_columns = data_latih.columns[:-1]
  #membuat dictionary kosong
  model = {}
  #menambahkan key prior yang berisikan nilai prior
  model['prior'] = prior
  #menambahkan key rata2 yang berisikan nilai rata2
  model['rata2'] = rata2
  #menambahkan key std yang berisikan nilai std
  model['std'] = std
  #menambahkan key list_kelas yang berisikan kelas pada data
  model['list_class'] = list_class
  #menambahkan key list_kolom yang berisikan kelas pada data
  model['list_columns'] = list_columns
  return model

Lakukanlah proses training dan simpan hasil training pada variabel **model**

In [155]:
#melakukan proses training dengan pemanggilan fungsi yang sudah dibuat 
model = training_naive_bayes(data_latih)

for key in model.keys():
  print("nilai", key)
  print(model[key])
  print()

nilai prior
Counter({'Iris-versicolor': 0.35833333333333334, 'Iris-virginica': 0.325, 'Iris-setosa': 0.31666666666666665})

nilai rata2
{('Iris-virginica', 'sepal_length'): 6.592307692307695, ('Iris-versicolor', 'sepal_length'): 5.976744186046511, ('Iris-setosa', 'sepal_length'): 5.0026315789473665, ('Iris-virginica', 'sepal_width'): 2.956410256410256, ('Iris-versicolor', 'sepal_width'): 2.7418604651162797, ('Iris-setosa', 'sepal_width'): 3.41842105263158, ('Iris-virginica', 'petal_length'): 5.5666666666666655, ('Iris-versicolor', 'petal_length'): 4.258139534883721, ('Iris-setosa', 'petal_length'): 1.4552631578947368, ('Iris-virginica', 'petal_width'): 1.994871794871794, ('Iris-versicolor', 'petal_width'): 1.3116279069767438, ('Iris-setosa', 'petal_width'): 0.24999999999999994}

nilai std
{('Iris-virginica', 'sepal_length'): 0.6941047302231982, ('Iris-versicolor', 'sepal_length'): 0.5317702732170143, ('Iris-setosa', 'sepal_length'): 0.36943114935120597, ('Iris-virginica', 'sepal_width'

## 6) Proses *testing*
Proses testing dilakukan dengan menghitung nilai posterior dari data uji berdasarkan nilai model yang diperoleh saat training. Nilai posterior diperoleh dari perkalian prior dengan *likelihood* masing-masing fiturnya. Setelah itu, penentuan kelas data uji dilakukan berdasarkan nilai posterior terbesar

In [156]:
#membuat dungsi testing naive bayes dengan argumen model yang sudah dilatih dan data uji
def testing_naive_bayes_gaussian(model,data_uji):
  #mengambil nilai prior pada model yang sudah dibuat
  prior = model['prior']
  #mengambil nilai rata2 pada model yang sudah dibuat
  rata2 = model['rata2']
  #mengambil nilai std pada model yang sudah dibuat
  std = model['std']
  #mengambil nilai kelas/label pada model yang sudah dibuat
  list_class = model['list_class']
  #mengambil nilai kolom pada data model yang sudah dibuat
  list_columns = model['list_columns']
  posterior = dict.fromkeys(list_class, 1)
  for a_class in list_class:
    for column in list_columns:
      #menghitung posterior
      posterior[a_class] = posterior[a_class]*hitung_likelihood_gaussian(data_uji[column],rata2[(a_class,column)],std[(a_class,column)])
      posterior[a_class] = posterior[a_class]*prior[a_class]
      #menentukan kelas uji berdasarkan nilai probabilitas tertinggi
  kelas_uji = max(posterior,key=posterior.get)
  return kelas_uji


Ujilah fungsi **testing_naive_bayes_gaussian** menggunakan data uji pertama. Bandingkan hasilnya dengan nilai label sebenarnya.

In [157]:
#melakukan pengujian pada indeks ke 0
indeks_uji = 0
#pemanggilan fungsi naive bayes untuk melakukan modelling
prediksi = testing_naive_bayes_gaussian(model,data_uji.iloc[indeks_uji])
print(prediksi)
print(label_uji.iloc[indeks_uji])

Iris-versicolor
Iris-versicolor


Lakukan pengujian untuk semua data uji. Selanjutnya bandingkan dengan label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar

In [158]:
#membuat list kosong
prediksi_total=[]
#melakukan perbandingan hasil uji dengan label sebenarnya
for indeks in range (data_uji.shape[0]):
  prediksi_total.append(testing_naive_bayes_gaussian(model,data_uji.iloc[indeks]))

#menghitung jumlah label yang sudah benar
print("total prediksi benar: ",sum(prediksi_total==label_uji))
print(data_uji.shape[0])
print(prediksi_total)

total prediksi benar:  30
30
['Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica']


## TUGAS
Pada tugas kali ini Anda akan menggunakan dataset **[Car Evaluation Dataset](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation)** yang telah dimodifikasi. Fitur-fitur yang ada bertipe kategori sebagai berikut:
1. **buying**. Merupakan harga beli mobil. Nilai : vhigh, high, med, low
2. **maint**. Menandakan biaya perawatan mobil. Nilai: vhigh, high, med, low
3. **lug_boot**. Menandakan ukuran bagasi. Nilai: small, med, big
4. **safety**. Menandakan skor keamanan mobil. Nilai:  low, med, high
5. **class**. Merupakan kelas data. Nilai: unacc, acc, good, vgood

Download dataset dari https://dataset-ppm.s3.amazonaws.com/car_sample.csv dan letakkan dalam variabel **data_tugas**.

In [159]:
#mengunduh dataset yang akan digunakan
!wget https://dataset-ppm.s3.amazonaws.com/car_sample.csv

--2021-07-09 13:55:26--  https://dataset-ppm.s3.amazonaws.com/car_sample.csv
Resolving dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)... 52.216.176.51
Connecting to dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)|52.216.176.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7226 (7.1K) [text/csv]
Saving to: ‘car_sample.csv.4’

car_sample.csv.4    100%[===================>]   7.06K  --.-KB/s    in 0s      

2021-07-09 13:55:26 (271 MB/s) - ‘car_sample.csv.4’ saved [7226/7226]



In [160]:
#melakukan import library yang akan digunakan
import pandas as pd #pemanggilan library pandas dengan alias pemanggilan pd
import numpy as np #pemanggilan library numpy dengan alias pemanggilan np

#melakukan pembacaan dataset yang sudah diunduh menggunakan pandas
data_tugas = pd.read_csv('car_sample.csv')
#menampilkan 5 data teratas
data_tugas.head()

,buying,maint,lug_boot,safety,class
0,vhigh,low,small,high,acc
1,vhigh,low,big,high,acc
2,vhigh,med,big,med,acc
3,med,med,med,high,acc
4,low,high,med,high,acc


Bagi data menjadi data latih dan uji dengan nama **tugas_latih** dan **tugas_uji** dengan rasio 70%:30%

In [161]:
#melakukan import sebagian library yang akan digunakan
#hanya mengimport train_test_split dari modul sklearn.model_selection
from sklearn.model_selection import train_test_split
#melakukan pembagian dataset dengan pembagian 30% data uji dan 70% data latih
tugas_latih, tugas_uji = train_test_split(data_tugas, test_size = 0.3, train_size = 0.7)

#cek jumlah baris data tugas latih 
print(tugas_latih.shape[0])

#cek jumlah baris data tugas uji
print(tugas_uji.shape[0])

182
78


Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya **tugas_label_uji**

In [162]:
#melakukan pemisahan antara fitur dan label dengan melakukan pop pada kolom class pada label uji
tugas_label_uji = tugas_uji.pop('class')

In [163]:
#menampilkan data uji yang hanya berisikan fitur data saja tanpa label
tugas_uji.head()

,buying,maint,lug_boot,safety
116,low,low,med,med
253,med,med,med,high
99,low,med,big,med
206,med,med,big,high
156,vhigh,vhigh,med,high


Hitunglah nilai prior dari data latih dan simpan nilainya pada variabel bernama **prior_tugas**. Anda dapat menggunakan fungsi **hitung_prior** yang telah Anda buat.

In [164]:
#menghitung nilai prior dari tugas_latih
prior_tugas = hitung_prior(tugas_latih['class'])
print(prior_tugas)

Counter({'unacc': 0.2692307692307692, 'acc': 0.25274725274725274, 'vgood': 0.24175824175824176, 'good': 0.23626373626373626})


Buatlah fungsi bernama **hitung_likelihood_multinomial** dengan parameter input berupa data latih. Fungsi ini bertujuan menghitung nilai likelihood pada masing-masing kolom pada data latih. Operasi-operasi yang perlu dilakukan pada fungsi ini adalah:


1.   Mendapatkan daftar kolom data
2.   Mendapatkan nama kolom kelas pada data (kolom terakhir pada data)
3.   Mendapatkan nama-nama kelas pada data (unik).
4.   Menghitung likelihood pada masing-masing kolom, nilai kolom, dan nilai kelas

Sebagai saran, Anda dapat menggunakan struktur data dictionary untuk menyimpan nilai likelihood. Key pada dictionary tersebut adalah tuple(nama kolom, nilai pada kolom, nama kelas). Petunjuk penggunaan dictionary dengan key berupa tuple dapat dibaca [di sini](https://stackoverflow.com/questions/1938614/in-what-case-would-i-use-a-tuple-as-a-dictionary-key).



In [165]:
def hitung_likelihood_multinomial(data_latih):
  #membuat list kolom yang berisi fitur dataset
  list_columns = data_latih.columns[:-1]
  #membuat list kolom yang berisi kelas/label dataset
  class_column_name = data_latih.columns[-1]
  #membuat set data yang berisikan kelas tiap data dan tidak ada duplikat
  list_class = set(data_latih[class_column_name])
  likelihood = {}
  for column in list_columns:
    for a_class in list_class:
      for value in set(data_latih[column]):
        #menghitung pembilang pada conditional probability (p(w|c)) yang menghitung kemunculan fitur w pada kategori c
        numerator = data_latih.loc[(data_latih[column]==value) & (data_latih[class_column_name]==a_class)][column].count()
        #menghitung total kemunculan semua fitur pada kategori c (count c)
        denumerator = data_latih.loc[data_latih[class_column_name]==a_class][column].count()
        #menghitung conditional probability (p(w|c))
        likelihood[(column,value,a_class)] = numerator/denumerator
  return likelihood

Ujilah fungsi **hitung_likelihood_multinomial** dengan parameter input berupa variabel **tugas_latih**

In [166]:
#menguji fungsi hitung_likelihood_multinomial dengan parameter tugas_latih
likelihood = hitung_likelihood_multinomial(tugas_latih)
likelihood

{('buying', 'high', 'acc'): 0.2608695652173913,
 ('buying', 'high', 'good'): 0.0,
 ('buying', 'high', 'unacc'): 0.24489795918367346,
 ('buying', 'high', 'vgood'): 0.0,
 ('buying', 'low', 'acc'): 0.13043478260869565,
 ('buying', 'low', 'good'): 0.6744186046511628,
 ('buying', 'low', 'unacc'): 0.24489795918367346,
 ('buying', 'low', 'vgood'): 0.6136363636363636,
 ('buying', 'med', 'acc'): 0.32608695652173914,
 ('buying', 'med', 'good'): 0.32558139534883723,
 ('buying', 'med', 'unacc'): 0.24489795918367346,
 ('buying', 'med', 'vgood'): 0.38636363636363635,
 ('buying', 'vhigh', 'acc'): 0.2826086956521739,
 ('buying', 'vhigh', 'good'): 0.0,
 ('buying', 'vhigh', 'unacc'): 0.2653061224489796,
 ('buying', 'vhigh', 'vgood'): 0.0,
 ('lug_boot', 'big', 'acc'): 0.3695652173913043,
 ('lug_boot', 'big', 'good'): 0.37209302325581395,
 ('lug_boot', 'big', 'unacc'): 0.2857142857142857,
 ('lug_boot', 'big', 'vgood'): 0.5681818181818182,
 ('lug_boot', 'med', 'acc'): 0.391304347826087,
 ('lug_boot', 'med'

Buatlah fungsi bernama **training_naive_bayes_multinomial** dengan input berupa data latih. Fungsi ini bertujuan membentuk model yang berisi:


1.   Nilai prior semua kelas
2.   Nilai likelihood semua kolom, nilai kolom, dan kelas
3.   Daftar kelas pada data latih (unik)
4.   Daftar nama kolom pada data latih

Return value dari fungsi ini adalah **model**



In [167]:
##membuat fungsi training naive bayes menggunakan algoritma multinomial
def training_naive_bayes_multinomial(data_latih):
  #membuat list kolom yang berisi kelas/label dataset
  class_column_name = data_latih.columns[-1]
  #menghitung probabilitas prior
  prior = hitung_prior(data_latih[class_column_name])
  #menghitung likelihood / conditional probability data latih
  likelihood = hitung_likelihood_multinomial(data_latih)
  #membuat set data yang berisikan kelas tiap data dan tidak ada duplikat
  list_class = set(data_latih[class_column_name])
  #mengambil semua fitur pada data latih
  list_columns = data_latih.columns[:-1]
  #membuat dictionary kosong
  model = {}
  #menambahkan key prior yang berisikan nilai prior
  model['prior'] = prior
  #menambahkan key likelihood yang berisikan nilai likelihood
  model['likelihood'] = likelihood
  #menambahkan key list_kelas yang berisikan kelas pada data
  model['list_class'] = list_class
  #menambahkan key list_kolom yang berisikan kelas pada data
  model['list_columns'] = list_columns
  return model

Jalankan fungsi **training_naive_bayes_multinomial** dan simpan hasilnya dalam variabel **model_tugas**

In [168]:
#menghitung model tugas dengan memanggil fungsi training_naive_bayes_multinomial yang sudah dibuat
model_tugas = training_naive_bayes_multinomial(tugas_latih)
model_tugas

{'likelihood': {('buying', 'high', 'acc'): 0.2608695652173913,
  ('buying', 'high', 'good'): 0.0,
  ('buying', 'high', 'unacc'): 0.24489795918367346,
  ('buying', 'high', 'vgood'): 0.0,
  ('buying', 'low', 'acc'): 0.13043478260869565,
  ('buying', 'low', 'good'): 0.6744186046511628,
  ('buying', 'low', 'unacc'): 0.24489795918367346,
  ('buying', 'low', 'vgood'): 0.6136363636363636,
  ('buying', 'med', 'acc'): 0.32608695652173914,
  ('buying', 'med', 'good'): 0.32558139534883723,
  ('buying', 'med', 'unacc'): 0.24489795918367346,
  ('buying', 'med', 'vgood'): 0.38636363636363635,
  ('buying', 'vhigh', 'acc'): 0.2826086956521739,
  ('buying', 'vhigh', 'good'): 0.0,
  ('buying', 'vhigh', 'unacc'): 0.2653061224489796,
  ('buying', 'vhigh', 'vgood'): 0.0,
  ('lug_boot', 'big', 'acc'): 0.3695652173913043,
  ('lug_boot', 'big', 'good'): 0.37209302325581395,
  ('lug_boot', 'big', 'unacc'): 0.2857142857142857,
  ('lug_boot', 'big', 'vgood'): 0.5681818181818182,
  ('lug_boot', 'med', 'acc'): 0.3

Buatlah fungsi **testing_naive_bayes_multinomial** untuk mendapatkan kelas dari sebuah data uji. Langkah yang dilakukan hampir sama dengan Gaussian, yaitu mengalikan likelihood semua kolom dengan posterior.

In [169]:
def testing_naive_bayes_multinomial(model,data_uji):
  #mengambil nilai prior pada model yang sudah dibuat
  prior = model['prior']
  #mengambil nilai likelihood pada model yang sudah dibuat
  likelihood = model['likelihood']
  #mengambil nilai kelas/label pada model yang sudah dibuat
  list_class = model['list_class']
  #mengambil nilai kolom pada data model yang sudah dibuat
  list_columns = model['list_columns']
  posterior = dict.fromkeys(list_class, 1)
  for a_class in list_class:
    for column in list_columns:
      #menghitung posterior
      posterior[a_class] = posterior[a_class]*likelihood[column,data_uji[column],a_class]
      #menentukan kelas uji berdasarkan nilai probabilitas tertinggi
  kelas_uji = max(posterior,key=posterior.get)
  return kelas_uji

Lakukan pengujian menggunakan sebuah data uji

In [170]:
#mendeklarasikan indeks yang akan di check
idx = 10
#mengambil data uji pada indeks ke 10
uji = tugas_uji.iloc[idx]
#mencari kelas uji dari data uji ke 10 menggunakan multinomial naive bayes
kelas_uji = testing_naive_bayes_multinomial(model_tugas,uji)
print(kelas_uji)
print(tugas_label_uji.iloc[idx])

good
good


Lakukan pengujian terhadap semua data uji.

In [171]:
#membuat list kosong
prediksi_total=[]
#melakukan perbandingan hasil uji dengan label sebenarnya
for indeks in range (data_uji.shape[0]):
  prediksi_total.append(testing_naive_bayes_gaussian(model,data_uji.iloc[indeks]))

#menghitung jumlah label yang sudah benar
print("total prediksi benar: ",sum(prediksi_total==label_uji))
print(data_uji.shape[0])
print(prediksi_total)

total prediksi benar:  30
30
['Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-virginica', 'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica']


In [172]:
#membuat list kosong
prediksi_total =[]
#melakukan perbandingan hasil uji dengan label sebenarnya
for indeks in range(tugas_uji.shape[0]):
  prediksi_total.append(testing_naive_bayes_multinomial(model_tugas,tugas_uji.iloc[indeks]))

prediksi = sum(prediksi_total==tugas_label_uji)
#menghitung jumlah label yang sudah benar
print("Total prediksi benar: ", prediksi)

#print banyak data yang diuji
print("Total prediksi : ",len(tugas_label_uji)) 

#print akurasi
print(prediksi/len(tugas_label_uji))

Total prediksi benar:  66
Total prediksi :  78
0.8461538461538461


In [173]:
len(tugas_label_uji)

78